
### Anti-Alpha Decay Monitoring System
# This system detects erosion of portfolio alpha (excess returns) in real-time
# Key components:
# 1. Live data ingestion
# 2. Alpha calculation using CAPM
# 3. Decay trend detection
# 4. Real-time alerts

In [ ]:
# %%
# Required Libraries
%pip install yfinance pandas numpy matplotlib seaborn scipy ipywidgets threading

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
from datetime import datetime


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %%
# Configuration
PORTFOLIO = {
    'AAPL': 0.4,    # Apple 40%
    'MSFT': 0.3,    # Microsoft 30%
    'GOOGL': 0.3    # Google 30%
}
BENCHMARK = 'SPY'  # S&P 500 ETF
RISK_FREE_RATE = 0.05  # Annual risk-free rate (5%)
MONITORING_INTERVAL = 60  # Seconds between updates
ALPHA_DECAY_THRESHOLD = -0.05  # Critical alpha decay level
ROLLING_WINDOW = 30  # Days for alpha calculation

In [ ]:
# %%
# Live Data Fetcher
def fetch_live_data():
    """Fetch real-time portfolio and benchmark data with error handling"""
    try:
        # Get portfolio data
        portfolio_value = 0
        for ticker, weight in PORTFOLIO.items():
            stock = yf.Ticker(ticker)
            data = stock.history(period='1d', interval='1m')
            if not data.empty:
                last_price = data['Close'].iloc[-1]
                portfolio_value += last_price * weight
            else:
                print(f"No data for {ticker}")
                return None, None
        
        # Get benchmark data
        benchmark = yf.Ticker(BENCHMARK)
        bm_data = benchmark.history(period='1d', interval='1m')
        if not bm_data.empty:
            benchmark_price = bm_data['Close'].iloc[-1]
        else:
            print(f"No data for benchmark {BENCHMARK}")
            return None, None
        
        return portfolio_value, benchmark_price
    
    except Exception as e:
        print(f"Data fetch error: {e}")
        return None, None


In [ ]:
# %%
# Alpha Calculation Engine
class AlphaTracker:
    def __init__(self, window=ROLLING_WINDOW):
        self.history = pd.DataFrame(columns=['timestamp', 'portfolio', 'benchmark'])
        self.window = window  # Rolling window in days
        self.lock = threading.Lock()
        
    def update(self, portfolio_val, benchmark_val):
        """Add new data point to history with thread safety"""
        if portfolio_val is None or benchmark_val is None:
            return
            
        with self.lock:
            new_row = pd.DataFrame({
                'timestamp': [pd.Timestamp.now()],
                'portfolio': [portfolio_val],
                'benchmark': [benchmark_val]
            })
            self.history = pd.concat([self.history, new_row], ignore_index=True)
            
    def calculate_returns(self):
        """Calculate daily returns from price data"""
        if len(self.history) < 2:
            return pd.DataFrame()
            
        df = self.history.copy()
        df['portfolio_return'] = df['portfolio'].pct_change()
        df['benchmark_return'] = df['benchmark'].pct_change()
        return df.dropna()
    
    def compute_alpha(self):
        """Compute rolling alpha using CAPM model"""
        returns_df = self.calculate_returns()
        if len(returns_df) < self.window + 1: 
            return None, returns_df
        
        # Daily risk-free rate
        daily_rf = (1 + RISK_FREE_RATE)**(1/252) - 1
        
        # Calculate excess returns
        returns_df['portfolio_excess'] = returns_df['portfolio_return'] - daily_rf
        returns_df['benchmark_excess'] = returns_df['benchmark_return'] - daily_rf
        
        # Rolling CAPM regression
        alphas = []
        betas = []
        for i in range(len(returns_df)):
            if i < self.window:
                alphas.append(np.nan)
                betas.append(np.nan)
                continue
                
            window = returns_df.iloc[i-self.window:i]
            X = window['benchmark_excess']
            y = window['portfolio_excess']
            
            # CAPM regression: y = alpha + beta*X
            beta, alpha, _, _, _ = stats.linregress(X, y)
            alphas.append(alpha)
            betas.append(beta)
        
        returns_df['alpha'] = alphas
        returns_df['beta'] = betas
        return returns_df.iloc[-1]['alpha'], returns_df

In [ ]:
# %%
# Decay Detection System
class AlphaDecayDetector:
    def __init__(self, threshold=ALPHA_DECAY_THRESHOLD):
        self.threshold = threshold
        self.decay_signals = []
        
    def check_decay(self, current_alpha, alpha_history):
        """Detect alpha decay patterns"""
        if current_alpha is None or alpha_history is None:
            return False
        
        # Critical level breach
        if current_alpha < self.threshold:
            return True
        
        # Negative trend detection (last 5 periods)
        if len(alpha_history) > 5:
            recent_alphas = alpha_history['alpha'].dropna().tail(5).values
            if len(recent_alphas) == 5 and np.all(np.diff(recent_alphas) < 0):
                return True
                
        return False

In [ ]:
# %%
# Real-time Dashboard
class LiveDashboard:
    def __init__(self):
        self.alpha_tracker = AlphaTracker()
        self.decay_detector = AlphaDecayDetector()
        self.alert_history = []
        self.running = False
        self.thread = None
        
        # Create output widgets
        self.alpha_output = widgets.Output()
        self.plot_output = widgets.Output()
        self.alert_output = widgets.Output()
        self.control_output = widgets.Output()
        
        # Create control buttons
        self.start_button = widgets.Button(description="Start Monitoring")
        self.stop_button = widgets.Button(description="Stop Monitoring")
        self.start_button.on_click(self.start_monitoring)
        self.stop_button.on_click(self.stop_monitoring)
        
        # Assemble dashboard
        self.dashboard = widgets.VBox([
            widgets.HTML("<h2>Anti-Alpha Decay Monitor</h2>"),
            widgets.HBox([
                self.start_button, 
                self.stop_button
            ]),
            widgets.HTML(f"<div>Monitoring Interval: {MONITORING_INTERVAL} seconds</div>"),
            widgets.HBox([self.alpha_output, self.alert_output]),
            self.plot_output
        ])
        
    def update_dashboard(self):
        """Refresh dashboard components"""
        with self.alpha_output:
            clear_output(wait=True)
            current_alpha, history = self.alpha_tracker.compute_alpha()
            if current_alpha is not None:
                status = "CRITICAL" if current_alpha < ALPHA_DECAY_THRESHOLD else "NORMAL"
                status_color = "red" if status == "CRITICAL" else "green"
                print(f"Current Alpha: {current_alpha:.6f}")
                print(f"Status: <span style='color:{status_color}'>{status}</span>", unsafe_allow_html=True)
                print(f"Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        with self.plot_output:
            clear_output(wait=True)
            if history is not None and 'alpha' in history.columns:
                plt.figure(figsize=(10, 5))
                
                # Smooth alpha values for better visualization
                smooth_alpha = history['alpha'].rolling(5, min_periods=1).mean()
                
                # Create plot
                plt.plot(history['timestamp'], smooth_alpha, 'b-', label='Portfolio Alpha')
                plt.fill_between(history['timestamp'], smooth_alpha, ALPHA_DECAY_THRESHOLD, 
                                where=(smooth_alpha < ALPHA_DECAY_THRESHOLD), 
                                color='red', alpha=0.3, label='Decay Zone')
                plt.axhline(y=0, color='k', linestyle='--', alpha=0.5)
                plt.axhline(y=ALPHA_DECAY_THRESHOLD, color='r', linestyle=':', label='Decay Threshold')
                
                # Formatting
                plt.title('Rolling Alpha Trend')
                plt.ylabel('Alpha')
                plt.xticks(rotation=45)
                plt.legend()
                plt.grid(True, linestyle='--', alpha=0.7)
                plt.tight_layout()
                plt.show()
        
        with self.alert_output:
            clear_output(wait=True)
            if self.alert_history:
                print("🔴 ALERT HISTORY:")
                for alert in self.alert_history[-3:]:
                    print(f"- {alert}")
            else:
                print("No alerts detected")
                
    def monitoring_loop(self):
        """Main monitoring loop"""
        while self.running:
            try:
                # Fetch new data
                portfolio_val, benchmark_val = fetch_live_data()
                
                # Update tracking
                self.alpha_tracker.update(portfolio_val, benchmark_val)
                current_alpha, history = self.alpha_tracker.compute_alpha()
                
                # Check for decay
                if current_alpha is not None:
                    if self.decay_detector.check_decay(current_alpha, history):
                        alert_msg = f"ALPHA DECAY DETECTED! Alpha: {current_alpha:.4f} at {pd.Timestamp.now()}"
                        if alert_msg not in self.alert_history:  # Avoid duplicates
                            self.alert_history.append(alert_msg)
            
                # Update dashboard
                self.update_dashboard()
                
            except Exception as e:
                print(f"Monitoring error: {e}")
                
            time.sleep(MONITORING_INTERVAL)
    
    def start_monitoring(self, b):
        """Start monitoring thread"""
        if not self.running:
            self.running = True
            self.thread = threading.Thread(target=self.monitoring_loop)
            self.thread.daemon = True
            self.thread.start()
            print("Monitoring started")
    
    def stop_monitoring(self, b):
        """Stop monitoring thread"""
        if self.running:
            self.running = False
            if self.thread is not None:
                self.thread.join(timeout=1.0)
            print("Monitoring stopped")
    
    def show(self):
        """Display the dashboard"""
        display(self.dashboard)
        self.update_dashboard()  # Initial update

In [ ]:
if __name__ == "__main__":
    dashboard = LiveDashboard()
    dashboard.show()

/tmp/ipykernel_3655/4236264327.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.history = pd.concat([self.history, pd.DataFrame([new_row])], ignore_index=True)
